In [ ]:
import datetime
import json
from collections import defaultdict

class Student:
    def __init__(self, name, id_no, stream):
        self.name = name
        self.id_no = id_no
        self.stream = stream
        self.book1 = None
        self.book2 = None
        self.book_no = 0

    def to_dict(self):
        return {
            "name": self.name,
            "id_no": self.id_no,
            "stream": self.stream,
            "book1": self.book1,
            "book2": self.book2,
            "book_no": self.book_no
        }

    @classmethod
    def from_dict(cls, data):
        student = cls(data["name"], data["id_no"], data["stream"])
        student.book1 = data["book1"]
        student.book2 = data["book2"]
        student.book_no = data["book_no"]
        return student

class Node:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None

class LibraryManagement:
    def __init__(self):
        self.root = None
        self.students = {}
        self.books = defaultdict(lambda: [0, 0])
        self.load_data()

    def insert_book(self, root, key):
        if root is None:
            return Node(key)
        if key < root.key:
            root.left = self.insert_book(root.left, key)
        elif key > root.key:
            root.right = self.insert_book(root.right, key)
        return root

    def insert(self, key):
        self.root = self.insert_book(self.root, key)

    def contains(self, root, key):
        if root is None:
            return False
        if key == root.key:
            return True
        if key < root.key:
            return self.contains(root.left, key)
        return self.contains(root.right, key)

    def delete(self, root, key):
        if root is None:
            return root
        if key < root.key:
            root.left = self.delete(root.left, key)
        elif key > root.key:
            root.right = self.delete(root.right, key)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            root.key = self.min_value(root.right)
            root.right = self.delete(root.right, root.key)
        return root

    def min_value(self, root):
        current = root
        while current.left is not None:
            current = current.left
        return current.key

    def inorder_print(self, root):
        if root:
            self.inorder_print(root.left)
            print(root.key)
            self.inorder_print(root.right)

    def print_tree(self, root, space=0, level=0):
        if root is None:
            return
        space += 5
        self.print_tree(root.right, space)
        print()
        for i in range(5, space):
            print(" ", end="")
        print(f"[{root.key}]")
        self.print_tree(root.left, space)

    def register_student(self):
        """Register a new student in the system."""
        name = input("Enter student name: ")
        id_no = int(input("Enter student ID: "))
        stream = input("Enter student stream: ")
        if id_no in self.students:
            print("Student with this ID already exists.")
        else:
            self.students[id_no] = Student(name, id_no, stream)
            print("Student registered successfully.")

    def save_data(self):
        """Save students and books data to files."""
        with open('students.json', 'w') as file:
            json.dump({id_no: student.to_dict() for id_no, student in self.students.items()}, file, indent=4)
        
        with open('books.json', 'w') as file:
            json.dump(self.books, file, indent=4)

    def load_data(self):
        """Load students and books data from files."""
        try:
            with open('students.json', 'r') as file:
                students_data = json.load(file)
                self.students = {int(id_no): Student.from_dict(data) for id_no, data in students_data.items()}
        except FileNotFoundError:
            self.students = {}
        
        try:
            with open('books.json', 'r') as file:
                self.books = defaultdict(lambda: [0, 0], json.load(file))
        except FileNotFoundError:
            self.books = defaultdict(lambda: [0, 0])

    def librarian_menu(self):
        while True:
            print("\n1. Add book")
            print("2. Delete book")
            print("3. Update book")
            print("4. Print Books Details")
            print("5. Print Books in-order")
            print("6. Print tree")
            print("7. Exit")

            choice = int(input("Enter your choice: "))

            if choice == 1:
                book_name = input("Enter name of book: ")
                quantity = int(input("Enter quantity of book: "))
                if self.contains(self.root, book_name):
                    print("Book already exists.")
                else:
                    self.insert(book_name)
                    self.books[book_name][0] += quantity
                    self.books[book_name][1] += quantity
            elif choice == 2:
                book_name = input("Enter name of book: ")
                if self.contains(self.root, book_name):
                    self.root = self.delete(self.root, book_name)
                    del self.books[book_name]
            elif choice == 3:
                book_name = input("Enter name of book: ")
                if self.contains(self.root, book_name):
                    quantity = int(input("Enter quantity of book to add more: "))
                    self.books[book_name][0] += quantity
                    self.books[book_name][1] += quantity
            elif choice == 4:
                for book_name, quantities in self.books.items():
                    print(f"Name of book: {book_name}")
                    print(f"Total Quantity: {quantities[0]}")
                    print(f"Available Quantity: {quantities[1]}")
                    print()
            elif choice == 5:
                self.inorder_print(self.root)
            elif choice == 6:
                self.print_tree(self.root)
            elif choice == 7:
                self.save_data()
                break

    def user_menu(self):
        while True:
            print("\n1. Issue book")
            print("2. Return book")
            print("3. Exit")

            choice = int(input("Enter your choice: "))

            if choice == 1:
                id_no = int(input("Enter your ID: "))
                student = self.students.get(id_no)
                if not student:
                    print("Student not found. Registering new student.")
                    self.register_student()
                    student = self.students[id_no]

                if student.book_no == 2:
                    print("You can't issue more than two books.")
                    continue

                book_name = input("Enter name of book: ")
                if self.contains(self.root, book_name) and self.books[book_name][1] > 0:
                    if student.book1 is None:
                        student.book1 = book_name
                    else:
                        student.book2 = book_name
                    self.books[book_name][1] -= 1
                    student.book_no += 1
                    issue_time = datetime.datetime.now()
                    due_time = issue_time + datetime.timedelta(seconds=5)
                    print(f"Book issued successfully. Due date: {due_time}")
                else:
                    print("Book is not available.")
            elif choice == 2:
                id_no = int(input("Enter your ID: "))
                student = self.students.get(id_no)
                if not student:
                    print("Student not found.")
                    continue

                book_name = input("Enter name of book: ")
                if (student.book1 != book_name and student.book2 != book_name):
                    print("This book is not issued to you.")
                    continue

                return_time = datetime.datetime.now()
                due_time = return_time - datetime.timedelta(seconds=5)  # Assuming due time is stored

                if return_time > due_time:
                    delay = (return_time - due_time).total_seconds()
                    fine = delay * 5
                    print(f"Book is overdue by {delay} seconds. Fine: {fine} Rs.")
                else:
                    print("Book returned successfully.")

                if student.book1 == book_name:
                    student.book1 = None
                else:
                    student.book2 = None

                self.books[book_name][1] += 1
                student.book_no -= 1
            elif choice == 3:
                self.save_data()
                break

    def main_menu(self):
        while True:
            print("\n1. Librarian Login")
            print("2. User Login")
            print("3. Exit")

            choice = int(input("Enter your choice: "))

            if choice == 1:
                user_id = input("Enter UserId: ")
                password = input("Enter Password: ")
                if user_id == "dsa@1" and password == "abc123":
                    print("Login successful.")
                    self.librarian_menu()
                else:
                    print("Invalid UserID or Password.")
            elif choice == 2:
                self.user_menu()
            elif choice == 3:
                self.save_data()
                break

if __name__ == "__main__":
    library_system = LibraryManagement()
    library_system.main_menu()
